In [184]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [185]:
from scipy.signal import medfilt

In [186]:
run /Users/charliewatson/Gdrive/SWRI/pteropractice/winview_v02.py

In [187]:
run /Users/charliewatson/Gdrive/SWRI/pteropractice/chzphot_v02.py

In [188]:
# Loading full set of darks
darks = load_spe('/Users/charliewatson/Gdrive/SWRI/occData/20150629UT/20150629UT_35.SPE')

xs, ys, numFrames, pixType =  256 256 3800 uint16


In [189]:
# Excluding the first frame of the darks, and taking the mean over the z-axis
darkM = mean(darks[1:,:,:], axis=0)

In [190]:
# Loading occultation data
roo = load_spe('/Users/charliewatson/Gdrive/SWRI/occData/20150629UT/20150629UT_34.SPE')

xs, ys, numFrames, pixType =  256 256 3800 uint16


In [191]:
# Subtracting the darks from the occ data
roo = roo - darkM
imshow(roo[1,:,:],interpolation='nearest')

In [192]:
# Loading flats
flat = load_spe('/Users/charliewatson/Gdrive/SWRI/occData/20150629UT/20150629UT_3.SPE')

xs, ys, numFrames, pixType =  512 512 100 uint16


In [193]:
# Taking a subframe of the flats (they were taken at 512x512 originally) and subtracting the darks
flat = flat[:,:256,:256] - darkM

In [194]:
# Taking the mean of the flats along the z-axis
flatM = mean(flat, axis=0)

In [195]:
# Normalize the flat using wizardry
flat = flatM/median(flatM)
imshow(flat,interpolation='nearest')
colorbar()

<matplotlib.colorbar.Colorbar instance at 0x10bb8e560>

In [196]:
# Taking the normalized flat out of the data
roo = roo/flat

In [197]:
clf()
#imshow(roo[1,:,:], interpolation='nearest')
imshow(mean(roo,axis=0), interpolation='nearest')

In [198]:
xShift = zeros(len(roo))
yShift = zeros(len(roo))
#for i in range (len(roo)-1):
    # use a fast fourier transform to find the cross correlation function for each subsequent frame
    #cc_fast = ifft2(fft2(roo[2000,:,:]) * conj(fft2(roo[i])))
    # find max indices of cc_fast and add those indices to the shift arrays
    #aaron = cc_fast.argmax()
    #yShift[i] = aaron // 256
    #xShift[i] = aaron % 256

In [217]:
xg,yg = 49,118
xc1,yc1 = zeros(len(roo)),zeros(len(roo))

In [218]:
for i in range(len(roo)):
    x,y = cntrd(roo[i,:,:],10,xg,yg)
    xc1[i],yc1[i] = x,y 
    

In [219]:
for i in range(len(roo)-1):
    xShift[i] = xc1[i+1] - xc1[i]
    yShift[i] = yc1[i+1] - yc1[i]

In [220]:
print(xShift)
print(yShift)
#print xShift
#for i in range (len(roo)-1):
    #roo[i+1,:,:] = roll(roo[i+1,:,:],int64(yShift[i]),axis=0)
    #roo[i+1,:,:] = roll(roo[i+1,:,:],int64(xShift[i]),axis=1)

[-0.52539436  0.44631752 -0.25729975 ...,  1.92052524 -1.72979463  0.        ]
[-1.83133427  0.03203067 -0.34421095 ...,  1.2933866  -1.43742182  0.        ]


In [221]:
#clf()
#imshow(mean(roo,axis=0),interpolation='nearest')

In [222]:
xc = zeros(len(roo))
yc = zeros(len(roo))
xg1 = 49
yg1 = 118
for i in range(len(xShift)):
    xc[i] = xg1 + sum(xShift[:i+1])
    yc[i] = yg1 + sum(yShift[:i+1])

In [223]:
# Use cen
# 119,50 first guess
#xc[0]=49
#yc[0]=118
'''
for i in range (len(roo)-1):
    if i == 0:
        x,y=cntrd(roo[i,:,:], 10, xc[0], yc[0])
        xc[i+1]=x
        yc[i+1]=y
    else:
        x,y=cntrd(roo[i,:,:], 10, xc[0]+xShift[i], yc[0]+xShift[i])
        xc[i+1]=x
        yc[i+1]=y 

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-223-5cb0fb04f479>, line 14)

In [224]:
flux = zeros(len(roo))
bgflux = zeros(len(roo))

# Use apPhot and some parameters to compute flux and bgflux
for i in range (len(roo)):
    fx,bfx = apPhot(roo[i,:,:],xc[i],yc[i],10.,14.,18.,28.)
    flux[i]=fx
    bgflux[i]=bfx

In [225]:
# The light curve! The holy grail
clf()
figure(2)
plot(flux)

In [227]:
figure(3)
clf()
plot(yShift)